In [1]:
import os, csv, json
import numpy as np

train_file = '../../datasets/metadata - training (temp).csv'
test_file = '../../datasets/metadata - testing (temp).csv'

dst_file = 'the fold.json'


In [2]:
# GET METADATA
meta = {
    'train': np.loadtxt(
        train_file,
        delimiter=',',
        dtype=np.str_
    ),
    'test': np.loadtxt(
        test_file,
        delimiter=',',
        dtype=np.str_
    )
}
accessions = {
    set_type: meta[set_type][:, 0]
    for set_type in meta
}
for set_type in accessions:
    assert len(np.unique(accessions[set_type])) == len(accessions[set_type])

assert len(set(accessions['train']) & set(accessions['test'])) == 0

# TRAIN AND VALIDATION ACCESSIONS
print('# tv accessions:', len(accessions['train']))
# TEST ACCESSIONS
print('# tv accessions:', len(accessions['test']))


# tv accessions: 20
# tv accessions: 10


In [3]:
# randomize
tv_accessions = np.char.add(np.random.permutation(accessions['train']), '-AFv6')

n_folds = 10
n_entries_per_fold = tv_accessions.size // n_folds

folds = [
    # tv_accessions[i:i+n_entries_per_fold].tolist()
    # for i in range(0, n_folds*n_entries_per_fold, n_entries_per_fold)
]
for fold_idx in range(n_folds):
    if fold_idx != n_folds - 1:
        fold = tv_accessions[
            fold_idx*n_entries_per_fold : (fold_idx+1)*n_entries_per_fold
        ]
    else:
        fold = tv_accessions[fold_idx*n_entries_per_fold:]
    folds.append(fold.tolist())

print(len(folds))


10


In [4]:
mini_fold = {
    'train': {},
    'valid': {}
}

for fold_idx in range(n_folds):

    # omit the fold_idx-th fold
    train_set = [
        acc
        for i, fold_entries in enumerate(folds)
        for acc in fold_entries
        if i != fold_idx
    ]
    valid_set = folds[fold_idx]

    mini_fold['train'][fold_idx] = train_set
    mini_fold['valid'][fold_idx] = valid_set

with open(dst_file, 'w') as f:
    json.dump(mini_fold, f, indent=2)
